# 部署机器学习模型

本实验将使用线性学习器算法训练并部署模型。您将在本笔记本中测试此模型。

然后使用 XGBoost 作为算法（使用和线性学习器算法相同的数据集）训练并部署模型。

部署之后，您将创建新的终端节点配置，这样做会将新的 XGBoost 模型添加为生产变体，而无需向其发送任何流量。然后，在准备好执行蓝/绿部署之后，您将更新终端节点配置中两个模型的权重，以从线性学习器模型“切换”到 XGBoost 模型。



本实验的前几个步骤涉及一个称为“特征工程”的功能。虽然我们对本实验中的核心数据集做了一些非常基本的调整，但请注意，如果这是一个真实世界的开发项目，我们可以并且应该执行一些其他步骤以使模型更准确。为简便起见，本实验省去了这些额外的步骤。



## 步骤 1 - 加载数据

我们使用西雅图 King Country 的一组住房数据进行训练。我们将这组数据放在 CSV 文件中，需要将其读取到 Pandas 数据框架“df”中。

数据加载后，我们调用 head 方法以查看前几行数据。

我们首先加载要在整个实验中使用的多个库和实用程序。

In [ ]:
# 加载需要的库和实用程序

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sagemaker
import boto3
import os
import re
from sklearn.model_selection import train_test_split
import time
import io 
import sagemaker.amazon.common as smac 
import os
import math
from sagemaker.amazon.amazon_estimator import image_uris
from sklearn import metrics 
from io import StringIO 
from sagemaker.inputs import TrainingInput
from sagemaker.amazon.amazon_estimator import image_uris
from sagemaker.serializers import CSVSerializer


In [ ]:
# 将数据集加载到初始数据框架 df
df = pd.read_csv("kc_house_data_2.csv")

# 查看前 5 行数据
df.head()

如您所见，该数据集有 21 列：
* `id` – 唯一 ID 编号
* `date` – 房屋出售日期
* `price` – 房屋售价
* `bedrooms` – 卧室数量
* `bathrooms` – 浴室数量
* `sqft_living` – 居住空间的平方英尺数
* `sqft_lot` – 地皮面积的平方英尺数
* `floors` – 房屋楼层数
* `waterfront` – 房屋是否位于海滨
* `view` – 可看到风景的面数
* `condition` – 房屋状况
* `grade` – 按建筑质量的分类 
* `sqft_above` – 地上面积的平方英尺数
* `sqft_basement` – 地下面积的平方英尺数
* `yr_built` – 建造年份
* `yr_renovated` – 装修年份
* `zipcode` – 邮政编码
* `lat` – 纬度
* `long` – 经度
* `sqft_living15` – 2015 年居住空间的平方英尺数（与最近修订版中的 `sqft_living` 不同）
* `sqrt_lot15` – 2015 年地皮面积的平方英尺数（与最近修订版中的 `sqft_lot` 不同）

这是要处理的优秀数据集，我们将对其执行大量特征工程，但本实验将专注于部署模型。但在这之前，我们必须先完成一些基本的特征工程步骤。首先将一些数据从数据框架 df 加载到一个新数据框架 data 中。加载完成后，我们可以保留原始数据框架，以防将来要返回到原始数据框架。

首先将居住空间的平方英尺数添加到新数据框架。

## 步骤 2 - 创建工作数据框架

首先将居住空间的平方英尺数添加到新数据框架。

In [ ]:
# 将居住空间的平方英尺数添加到称为 'data' 的新数据框架中
data = df[['sqft_living']].copy()

接下来，添加无需转换或“设计”的其他特征。

In [ ]:
data['bedrooms'] = df['bedrooms']
data['bathrooms'] = df['bathrooms']
data['sqft_lot'] = df['sqft_lot']
data['floors'] = df['floors']

## 步骤 3 - 基本特征工程

### 分类变量

我们先来看一些分类特征，从简单的二元变量开始。

该数据集有 `waterfront` 特征，这是一个二进制变量。我们应该将编码从 `'Y'` 和 `'N'` 更改为 `1` 和 `0`。

该操作可以使用 Pandas 提供的 `map` 函数（[文档](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html)）来完成。这需要有一个应用于该列的函数，或一个用于查找正确转换的字典。

In [ ]:
# 使用 map 函数添加海滨信息作为二进制分类变量。

data['waterfront'] = df['waterfront'].map({'Y':1, 'N':0})

您还可以对许多类分类变量进行编码。查看 `condition` 列，该列提供了房屋质量的分数。看看[数据源] (https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r#b)，可以发现状况可以被视为有序分类变量，因此按顺序对其进行编码是可行的。

> 使用与上文相同的映射方法，我们将有序分类变量 `condition` 编码为从 1 到 5 的数值范围。

In [ ]:
# 使用 map 函数添加条件作为有序分类变量。

data['condition'] = df['condition'].map({'Poor':1, 'Fair':2, 'Average':3, 'Good':4, 'Very Good':5})

我们现在还将使用独热编码将一些名义分类特征转换为二进制。

In [ ]:
# 独热编码
data = pd.concat([data, pd.get_dummies(df['zipcode'])], axis=1)

# 扩展较大的特征
sqft_min = data['sqft_living'].min()
sqft_max = data['sqft_living'].max()
data['sqft_living'] = data['sqft_living'].map(lambda x : (x-sqft_min)/(sqft_max - sqft_min))

sqft_min2 = data['sqft_lot'].min()
sqft_max2 = data['sqft_lot'].max()
data['sqft_lot'] = data['sqft_lot'].map(lambda x : (x-sqft_min2)/(sqft_max2 - sqft_min2))

cond_min = data['condition'].min()
cond_max = data['condition'].max()
data['condition'] = data['condition'].map(lambda x : (x-cond_min)/(cond_max - cond_min))

In [ ]:
data.head()

可用于查看数据的另一种方法是 describe 方法。此方法可提供有关特征的一些基本统计信息。

In [ ]:
data.describe()

现在我们将设置训练数据，拆分并将训练数据转换为 RecordIP 格式，以便用于训练我们的线性学习器模型。

In [ ]:
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# 用于保存代码和模型构件的 S3 存储桶。
bucket = sagemaker.Session().default_bucket()
prefix = "linear-learner" #前缀是 S3 存储桶中的子文件夹/键
output_location = 's3://{}/{}/output'.format(bucket, prefix)

In [ ]:
# 拆分训练、验证和测试数据
ys = np.array(df['price']).astype("float32")
xs = np.array(data).astype("float32")

np.random.seed(8675309)
train_features, test_features, train_labels, test_labels = train_test_split(xs, ys, test_size=0.2)
val_features, test_features, val_labels, test_labels = train_test_split(test_features, test_labels, test_size=0.5)

In [ ]:
#创建一个 SageMaker 会话
sagemaker_session = sagemaker.Session()

#需要将数据集转换为 RecordIO 格式以便线性学习器理解

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, train_features, train_labels)
buf.seek(0) 

###上传训练数据
#上传到 S3 的训练数据的文件名 
key = 'linear-train-data'
#将训练数据上传到 S3
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_train_data))

###上传测试数据
buf = io.BytesIO() # 创建一个内存字节数组（buf 是我将数据写入的缓冲区）
smac.write_numpy_to_dense_tensor(buf, test_features, test_labels)
buf.seek(0)

#测试数据的子文件夹
key = 'linear-test-data'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test', key)).upload_fileobj(buf)
s3_test_data = 's3://{}/{}/test/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_test_data))

###模型构件
output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('Training artifacts will be uploaded to: {}'.format(output_location))

尽管我们可以使用更多特征并进行特征工程，但我们现在要停止这项任务，而要继续训练模型。

## 步骤 4 - 使用线性学习器算法训练模型


In [ ]:
# 设置存储线性学习器算法的容器
container1 = image_uris.retrieve('linear-learner', boto3.Session().region_name)

# 创建估算器
linear_model = sagemaker.estimator.Estimator(container1,
                                       role,
                                       instance_count = 1,
                                       instance_type = 'ml.m4.xlarge',
                                       output_path = output_location,
                                       sagemaker_session = sagemaker_session)

# 设置超参数
linear_model.set_hyperparameters(feature_dim = 77,
                               predictor_type = 'regressor',
                               mini_batch_size = 20,
                               epochs = 5,
                               num_models = 10,
                               loss = 'absolute_loss')


In [ ]:
# 现在我们可以传入之前声明的 S3 training_data 路径变量，然后训练第一个模型
linear_model.fit({'train': s3_train_data})

## 步骤 5 - 部署线性学习器模型

我们现在将部署使用线性学习器构建的初始模型。 

首先设置终端节点名称和模型名称变量，然后可以直接使用 .deploy() 方法部署模型。这会创建一个由 SageMaker 托管的新终端节点配置和终端节点。

稍后我们在本实验中开始部署 XGBoost 模型时，将使用相同的流程。

In [ ]:
# 设置变量名称
endpoint_name = 'home-price-regressor-endpoint'
linear_model_name = 'linear-regressor-model'

#部署初始模型

home_price_regressor = linear_model.deploy(initial_instance_count = 1,
                                           instance_type = 'ml.m4.xlarge',
                                           endpoint_name= endpoint_name,
                                           model_name= linear_model_name
                                          )



In [ ]:
# 我们需要确保数据采用对已部署模型而言正确的格式
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
home_price_regressor.serializer = CSVSerializer()
home_price_regressor.deserializer = JSONDeserializer()

## 步骤 6 - 与线性学习器模型交互

在真实世界场景中，这是我们在首次部署模型时发挥作用的一步。由于我们已部署了正常运行的模型，我们现在会将测试数据推送到该模型，然后通过散点图绘制其工作情况。


In [ ]:
result1 = home_price_regressor.predict(test_features)
result1 #应为 JSON

#迭代结果 JSON 以获得所有预测结果的 NP 数组，这样我们就能与 Y 测试做比较
predictions = np.array([res['score'] for res in result1['predictions']])
#预测 #现在应为 numpy 数组

#可视化预测相对于 y_test 的准确程度
plt.scatter(test_labels, predictions)

如您所见，预测结果很准确。如果在真实世界场景中，我们将返回并执行更多特征工程。但是，就本实验而言，我们假设预测结果满足我们的业务需求，因此我们开始使用此模型。

### 场景：  

运行模型一段时间后，我们觉得使用其他算法或许能做得更好。替换当前模型的第一步是训练一个新模型。如果由于概念漂移或其他事件而需要更新模型，但不想中断生产环境，则也可能会训练新模型。

## 步骤 7 - 训练新模型

现在我们将使用相同信息但使用 XGBoost 算法训练新模型。XGBoost 要求输入采用 CSV 格式，而不是像我们在上面所做的那样直接从数据框架提取数据。因此，我们需要设置训练、测试和验证数据，然后将其以 CSV 格式导出到 S3 存储桶。

In [ ]:
# 创建训练、测试和验证数据集
XX_train = pd.concat((pd.DataFrame(train_labels),pd.DataFrame(train_features)),axis = 1)
XX_valid = pd.concat((pd.DataFrame(val_labels),pd.DataFrame(val_features)),axis = 1)
XX_test = pd.concat((pd.DataFrame(test_labels),pd.DataFrame(test_features)),axis = 1)

In [ ]:
# 重新格式化 csv 文件，然后创建输入集
csv_buffer = StringIO()

bucket = sagemaker.Session().default_bucket()
prefix_out = 'output'
output_path = 's3://{}/{}/{}'.format(bucket, prefix_out, 'houseing-xgb')

XX_train.to_csv(csv_buffer,header=False, index=False)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'train.csv').put(Body=csv_buffer.getvalue())

XX_valid.to_csv(csv_buffer,header=False, index=False)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'valid.csv').put(Body=csv_buffer.getvalue())

XX_test.to_csv(csv_buffer,header=False, index=False)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'test.csv').put(Body=csv_buffer.getvalue())

s3_input_train = TrainingInput(s3_data='https://{}.s3.amazonaws.com/train.csv'.format(bucket), content_type='csv')
s3_input_validation = TrainingInput(s3_data='https://{}.s3.amazonaws.com/valid.csv'.format(bucket),content_type='csv')
s3_input_test = TrainingInput(s3_data='https://{}.s3.amazonaws.com/test.csv'.format(bucket),content_type='csv')


In [ ]:
# 现在设置容器、估算器和超参数，就像我们对第一个模型所做的那样
container = sagemaker.image_uris.retrieve('xgboost',boto3.Session().region_name,'latest')

data_channels = {'train': s3_input_train, 'validation': s3_input_validation, 'test': s3_input_test}

sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role=sagemaker.get_execution_role(),
                                    train_instance_count=1,
                                    train_instance_type='ml.m4.xlarge',
                                    output_path=output_path,
                                    sagemaker_session=sess)

xgb.set_hyperparameters(predictor_type='regressor',
                        max_depth=200,
                        num_round=100)

# 最后训练新模型
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation, 'test': s3_input_test})


## 步骤 8 - 部署新模型

我们现在将使用自己的终端节点和终端节点配置部署新模型。该模型最初是独立存在的，因此我们可以在需要时单独对其测试。我们这次在本实验中不会测试该模型。

In [ ]:
# 设置变量，然后再次使用 .deploy() 方法部署模型
xgb_endpoint_name = 'xgb-regressor-endpoint'
xgb_model_name = 'xgb-regressor-model'

xgb_price_regressor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge',
                           endpoint_name = xgb_endpoint_name,
                           model_name = xgb_model_name,
                          )

xgb_price_regressor.serializer = CSVSerializer()
xgb_price_regressor.deserializer = JSONDeserializer()


此时，我们已有托管线性学习器模型的原始生产终端节点和终端节点配置。我们还拥有针对 XGBoost 模型的单独终端节点和终端节点配置。但是，我们并不想将所有生产流量都切换到这个单独的终端节点，因此我们先更新第一个，以便完成蓝/绿部署。

# 步骤 9 - 创建新的终端节点配置
这个新的终端节点配置包括原始模型和新模型，将它们用作两个单独的生产变体。我们还会将初始权重设置为仅将流量发送到原始模型，以确保生产流量没有变化。

In [ ]:
# 获取当前终端节点配置
sage_client = sess.sagemaker_client
endpoint = sage_client.describe_endpoint(EndpointName=home_price_regressor.endpoint_name)
endpoint_config = sage_client.describe_endpoint_config(
    EndpointConfigName=endpoint['EndpointConfigName'])

# 将当前部署权重设置为 1，以确保所有流量像以前一样继续流入同一模型
current_model_config = endpoint_config['ProductionVariants'][0]
current_model_config['InitialVariantWeight'] = 1
current_model_config['VariantName'] = 'linear-learner'

# 现在设置一个新变体并为该变体设置配置
Variant = 'xgboost'

xgb_model_config = {'ModelName': xgb_model_name,
                      'InitialInstanceCount': 1,
                      'InstanceType': 'ml.m4.xlarge',
                      'VariantName': Variant,
                      'InitialVariantWeight': 0}

# 现在使用上文的两个生产变体创建新的终端节点配置。
sage_client.create_endpoint_config(
    EndpointConfigName='AB-Config',
    ProductionVariants=[current_model_config,
                        xgb_model_config])


In [ ]:
# 现在我们使用新的终端节点配置更新原始终端节点 - 此操作可能需要几分钟
sage_client.update_endpoint(
    EndpointName=endpoint['EndpointConfigName'],
    EndpointConfigName='AB-Config')

result = sess.wait_for_endpoint(endpoint['EndpointConfigName'])

此时，所有内容和以前一样处理，唯一的区别是我们的终端节点配置中有两个生产变体。

需要进行切换时，我们执行：

# 步骤 10 - 蓝/绿部署
只需更改终端节点配置中的权重，我们即可将所有流量从线性学习器模型切换到 XGBoost 模型

In [ ]:
sage_client.update_endpoint_weights_and_capacities(
    EndpointName=endpoint['EndpointConfigName'],
    DesiredWeightsAndCapacities=[
        {
            'VariantName': 'linear-learner',
            'DesiredWeight': 0
        },
        {
            'VariantName': 'xgboost',
            'DesiredWeight': 1
        }
    ]
)
response = sess.wait_for_endpoint(endpoint['EndpointConfigName'])

## 总结

您现在已训练并部署了两个不同的模型，然后执行了蓝/绿部署以从一个模型切换到另一个模型。

### 清理

如果您已用完此笔记本，请运行以下单元格删除托管终端节点，以免未关闭的闲置实例产生任何费用。

In [ ]:
sagemaker.Session().delete_endpoint(xgb_price_regressor.endpoint_name)
sagemaker.Session().delete_endpoint(home_price_regressor.endpoint_name)